In [1]:
import pandas as pd
import numpy as np
import nltk
import random
import time
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from copy import copy, deepcopy
import collections
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from pandas.tseries.offsets import *
import re
import collections
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import svm
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from os import path
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from wordcloud import WordCloud, STOPWORDS

porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

/Users/MaxTan/anaconda/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
data_ads = "/Users/MaxTan/Documents/CU_16fall/BDA/Project/Data/"

user_raw = pd.read_csv(data_ads + "wow3_user2.csv", names = ["user_id", "review_count", "average_stars", 
                                               "friends", "fans", "votes_cool", "votes_funny", 
                                               "votes_useful", "num", "degree", "coefficient"])

business_raw = pd.read_csv(data_ads+"wow3_business_mysql.csv", names = ["business_id", "name", "latitude", 
                                                           "longitude", "stars", "categories", 
                                                           "review_count", "open"])

review_raw = pd.read_csv(data_ads+"wow3_review_mysql.csv", names = ["review_id", "user_id", "business_id", 
                                                       "stars", "text", "date", "votes_funny", 
                                                       "votes_useful"])

all_raw = pd.read_csv(data_ads+"wow3_all2.csv", names = ["business_id", 
                                                       "name", "stars_business","business_review_count",
                                                       "categories", "open", "review_id", 
                                                       "stars_review", "text", "date", "votes_funny", 
                                                       "votes_useful","user_id", "user_review_count", 
                                                       "average_stars","friends", "fans", "num", 
                                                       "degree", "coefficient"])


/Users/MaxTan/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#2.Topic Model and Sentimental Analysis by Using Review Text:

In [4]:
# Sentimental Analysis (Random Forest Classification):
review_pos = review_raw[(review_raw['stars']==5) | (review_raw['stars']=='5')]

review_neg = review_raw[(review_raw['stars']<=2) | (review_raw['stars']=='1')|(review_raw['stars']=='2')]

traindf = review_raw[(review_raw['stars']==5) | (review_raw['stars']=='5')|(review_raw['stars']<=2) | (review_raw['stars']=='1')|(review_raw['stars']=='2')]

traindf['sentiment'] = pd.Series([0]*len(traindf),index = traindf.index)

for i in traindf.index:
    star = traindf.loc[i]['stars']
    if star == 5 or star == '5':
        traindf.set_value(i, 'sentiment', 1)

/Users/MaxTan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
pos_length, neg_length = len(traindf[traindf['sentiment']==1]),len(traindf[traindf['sentiment']==0])
trainpos = traindf[traindf['sentiment']==1].iloc[:int(pos_length*0.8)]
trainneg = traindf[traindf['sentiment']==0].iloc[:int(neg_length*0.8)]
testpos = traindf[traindf['sentiment']==1].iloc[int(pos_length*0.8):]
testneg = traindf[traindf['sentiment']==0].iloc[int(neg_length*0.8):]
frames = [trainpos,trainneg]
traindata = pd.concat(frames)
frames = [testpos,testneg]
testdata = pd.concat(frames)
traindata['review_text'] = pd.Series(traindata.text,index = traindata.index)
testdata['review_text'] = pd.Series(testdata.text,index = testdata.index)

In [6]:
# Unigram Set:
start_time  = time.time()
fdist = nltk.FreqDist()
for i in traindata.index:
    ly = traindata.text[i].lower()
    ly = ly.decode('utf-8')
    #1.Tokenization:
    try:
        tokens = nltk.word_tokenize(ly)
    except UnicodeDecodeError:
        continue
    #2.Special Characters Removed:
    tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
    tokens_lem_stem = []
    for word in tokens:
        try:
            #3.First Lemmatization, 4.then Stemming:
            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
        except UnicodeDecodeError:
            continue
    #5.Remove Punctuation:
    word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
    #6.Remove Stop Words:
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]
    fdist = fdist.__add__(nltk.FreqDist(word_stop))
print("--- %s seconds ---" % (time.time() - start_time))

fdist_uni = fdist

--- 4973.09739804 seconds ---


In [7]:
start_time = time.time()

num_unigram = 5000
# create features for training data
feat_uni = [i for [i,j] in fdist_uni.most_common(num_unigram)]
for unigram in feat_uni:
    traindata[unigram] = pd.Series([0.0]*len(traindata),index = traindata.index)
#create bigram features for training set:

#Try to use unigram to analyze and obtain the feature:
fdist = nltk.FreqDist()
for i in traindata.index:
    ly = traindata.review_text[i].lower()
    
    ly = ly.decode('utf-8')
    #1.Tokenization:
    try:
        tokens = nltk.word_tokenize(ly)
    except UnicodeDecodeError:
        print ly
        continue
        
    #2.Special Characters Removed:
    tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
    tokens_lem_stem = []
    for word in tokens:
        try:
            #3.First Lemmatization, 4.then Stemming:
            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
        except UnicodeDecodeError:
            continue

    #5.Remove Punctuation:
    word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
    #6.Remove stop words:
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]
    
    
    #compute frequency distribution for all the unigram in one song:
    fdist = nltk.FreqDist(word_stop)
    for unigram in feat_uni:
        traindata.set_value(i, unigram, fdist[unigram])
        #if unigram in word_stop:
        #    traindata.set_value(i, unigram, 1)



#change features for test data
for unigram in feat_uni:
    testdata[unigram] = pd.Series([0.0]*len(testdata),index = testdata.index)


#Try to use unigram to analyze and obtain the feature:
fdist = nltk.FreqDist()
for i in testdata.index:
    ly = testdata.review_text[i].lower()
    ly = ly.decode('utf-8')
    #1.Tokenization:
    try:
        tokens = nltk.word_tokenize(ly)
    except UnicodeDecodeError:
        print ly
        break
        
    #2.Special Characters Removed:
    tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
    tokens_lem_stem = []
    for word in tokens:
        try:
            #3.First Lemmatization, 4.then Stemming:
            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
        except UnicodeDecodeError:
            continue

    #5.Remove Punctuation:
    word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
    #6.Remove stop words:
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]    
    
    
    #compute frequency distribution for all the unigram in one song:
    fdist = nltk.FreqDist(word_stop)
    for unigram in feat_uni:
        testdata.set_value(i, unigram, fdist[unigram])
        #if unigram in word_stop:
        #    testdata.set_value(i, unigram, 1)    
print("--- %s seconds ---" % (time.time() - start_time))

--- 2969.48254895 seconds ---


In [8]:
start_time = time.time()

clf = RandomForestClassifier(n_estimators= 500,criterion="entropy",oob_score=True)#,min_samples_leaf=15)
clf = clf.fit(traindata[feat_uni],traindata['sentiment'])

print 'Correct oob rate of training set:',clf.oob_score_
print 'Correct rate of test set:',clf.score(testdata[feat_uni],testdata['sentiment'])


"""
res = clf.predict(testdata[feat_uni])
for i in set(testdata.sentiment.values):
    print 'Results of test set with '+str(i)+' decades difference :',sum(map(abs,res - testdata['sentiment'])==i)/float(len(res))
"""

print("--- %s seconds ---" % (time.time() - start_time))

Correct oob rate of training set: 0.923025765344
Correct rate of test set: 0.923895323895
--- 1425.03806591 seconds ---


In [9]:
start_time = time.time()

clf = RandomForestClassifier(n_estimators= 500,criterion="entropy",oob_score=True)#,min_samples_leaf=15)
clf = clf.fit(traindata[feat_uni],traindata['sentiment'])

#print 'Correct rate of training set:',clf.score(traindata[feat_uni],traindata['sentiment'])
print 'Correct oob rate of training set:',clf.oob_score_
print 'Correct rate of test set:',clf.score(testdata[feat_uni],testdata['sentiment'])


"""
res = clf.predict(testdata[feat_uni])
for i in set(testdata.sentiment.values):
    print 'Results of test set with '+str(i)+' decades difference :',sum(map(abs,res - testdata['sentiment'])==i)/float(len(res))
"""

print("--- %s seconds ---" % (time.time() - start_time))

Correct oob rate of training set: 0.922618153734
Correct rate of test set: 0.922865722866
--- 1407.38278103 seconds ---


In [11]:
#Methods 2: Naive Bayes:
gnb = GaussianNB()
y_pred = gnb.fit(traindata[feat_uni],traindata['sentiment']).predict(traindata[feat_uni])
print "Correct rate of train set:", float((traindata['sentiment'].values == y_pred).sum())/len(traindata)

y_pred = gnb.fit(traindata[feat_uni],traindata['sentiment']).predict(testdata[feat_uni])
print "Correct rate of test set:", float((testdata['sentiment'].values == y_pred).sum())/len(testdata)

Correct rate of train set: 0.852702035913
Correct rate of test set: 0.827456027456
